# 추천 시스템
1. 협업 필터링
- 다수의 사용자의 아이템 구매 이력 정보만으로 사용자간 유사성 및 아이템 간 유사성을 파악
-  아이템과 사용자 간의 행동 또는 관계에만 주목할 뿐 아이템 자체의 고유한 속성에 주목하지 않음
2. 콘텐츠 기반 필터링
- 아이템의 고유의 정보를 바탕으로 아이템 간 유사성을 파악
- 아이템 자체의 속성에만 주목하고 사용자와 아이템 간의 관련성에는 주목하지 않음

# 명시적 평가
- 좋아요나 별점와 같은 선호도

# 암묵적 평가
- 서비스를 사용하면서 자연스럽게 발생하는 암묵적(implicit)인 피드백

# 데이터셋
- Movielens 데이터
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있음 MovieLens 1M Dataset 사용을 권장
- 별점 데이터는 대표적인 explicit 데이터이지만 implicit 데이터로 간주하고 테스트해 볼 수 있음
- 별점을 시청횟수로 해석
- 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외

# 1. 데이터 준비와 전처리

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head(10)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [6]:
# 평점과 제목 데이터프레임 합치기
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [7]:
# 사용하지 않는 컬럼 제거
ratings = ratings.drop(columns=['timestamp', 'genre'])
ratings.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


# 2. 데이터 분석

In [8]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [9]:
# ratings에 있는 유니크한 영화 제목 수
ratings['title'].nunique()

3628

In [10]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [11]:
# 가장 인기 있는 영화 30개
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [12]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

# 3. 내가 선호하는 영화 추가

In [13]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('happy', regex=False)]

,movie_id,title,genre
102,104,Happy Gilmore (1996),Comedy
639,644,Happy Weekend (1996),Comedy
2822,2891,"Happy, Texas (1999)",Comedy
3246,3315,Happy Go Lovely (1951),Musical


In [14]:
# 내가 좋아하는 영화 5편의 id와 title 리스트 만들기
my_favorite_id = ['3404', '1', '1682', '586', '2762']
my_favorite_title = ['Titanic (1997)' , 'Toy Story (1995)' ,'Truman Show, The (1998)' ,'Home Alone (1990)' ,'Sixth Sense, The (1999)']

# 'jenny'라는 user_id가 위 5편의 영화를 5번씩 봤다고 가정
my_movielist = pd.DataFrame({'user_id': ['jenny']*5, 'movie_id': my_favorite_id, 'count': [5]*5, 'title': my_favorite_title})
my_movielist

,user_id,movie_id,count,title
0,jenny,3404,5,Titanic (1997)
1,jenny,1,5,Toy Story (1995)
2,jenny,1682,5,"Truman Show, The (1998)"
3,jenny,586,5,Home Alone (1990)
4,jenny,2762,5,"Sixth Sense, The (1999)"


In [15]:
if not ratings.isin({'user_id':['jenny']})['user_id'].any():  # user_id에 'jenny'라는 데이터가 없다면
    ratings = ratings.append(my_movielist, ignore_index=True)   # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,jenny,3404,5,Titanic (1997)
836479,jenny,1,5,Toy Story (1995)
836480,jenny,1682,5,"Truman Show, The (1998)"
836481,jenny,586,5,Home Alone (1990)
836482,jenny,2762,5,"Sixth Sense, The (1999)"


# 4. indexing 후 CSR matrix 생성

In [16]:
# 고유한 유저와 영화의 개수
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 고유한 값의 개수만큼 유저 id와 영화 제목을 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [17]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,3404,5,27
836479,6039,1,5,40
836480,6039,1682,5,385
836481,6039,586,5,507


In [18]:
from scipy.sparse import csr_matrix

# CSR Matrix 생성
num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5. AlternatingLeastSquares 모델 훈련시키기

In [19]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [21]:
# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 6. 훈련된 모델이 예측한 나의 선호도 파악

In [23]:
# 훈련된 모델이 만든 나의 벡터와 영화 벡터 구하기
jenny, titanic = user_to_idx['jenny'], movie_to_idx['Titanic (1997)']
jenny_vector, titanic_vector = als_model.user_factors[jenny], als_model.item_factors[titanic]

In [24]:
jenny_vector

array([ 0.22430515,  0.17618869, -0.6691161 ,  0.97457343, -0.17608222,
        0.8075873 , -0.3021627 , -0.32576367, -0.19819802, -0.31356815,
       -0.0906718 ,  0.6598122 , -0.13994437, -0.6603704 , -0.56656414,
        0.4369003 ,  0.40960336, -0.5479577 , -1.1902001 ,  0.9769281 ,
       -0.44432303,  0.45421672,  0.13773352,  0.14894769,  0.28756678,
        0.777767  , -0.08669937,  0.3794692 , -1.0260193 ,  0.20687294,
        0.9769322 ,  0.77625304,  0.18239821, -0.4400936 ,  0.17369905,
        0.05602873,  0.44858778,  0.0249983 ,  0.21382478,  0.08409972,
       -0.6903213 , -0.5327605 , -0.9527088 , -0.8644769 ,  0.47433954,
       -0.22294797,  0.3997604 ,  0.00636849, -0.43039185,  0.18103476,
       -0.68476945,  0.15028484,  0.8294395 ,  0.39347103,  0.21207045,
        0.17733374, -0.17726314,  0.2349217 ,  0.8275934 , -0.08061446,
        0.5330021 ,  0.5609189 , -0.09951682, -0.10520467, -0.25933304,
        0.23708516,  0.90822816, -0.35760653,  0.76988554,  0.34

In [25]:
titanic_vector   # 영화 타이타닉의 벡터

array([ 1.4334379e-02,  1.2988924e-03, -2.6735742e-02, -1.2741253e-03,
        4.4695903e-02,  3.0073904e-02,  8.2859304e-03, -2.8947478e-02,
       -4.9338187e-03, -1.8764181e-02, -7.2373389e-03,  4.3879885e-02,
       -2.3459287e-02, -1.8840311e-02,  4.6434123e-03,  6.8252780e-03,
        1.0521811e-02, -8.0114724e-03, -1.5447078e-03,  4.3332849e-02,
        2.0630222e-02,  1.0901998e-03,  1.9397378e-02, -1.0935541e-02,
       -1.7009841e-02,  1.0176537e-03, -9.4603011e-03,  6.4675640e-03,
       -1.5129714e-04, -2.9739385e-02,  2.7907021e-02,  3.3202440e-02,
        2.9360132e-02, -1.2418005e-02,  2.3703862e-03,  4.0498313e-02,
        2.2373863e-02, -1.5935270e-02,  1.0690268e-02, -1.5368544e-02,
       -3.2211810e-02, -9.5716836e-03,  8.7086931e-03, -7.1087210e-03,
        2.8084777e-02, -1.3243903e-02,  3.3424396e-02, -1.0372459e-02,
       -9.0304026e-03,  4.2970616e-02,  5.9769424e-03,  1.9377794e-02,
        5.5009252e-03,  2.1587998e-02,  8.2574943e-03,  1.0689802e-02,
      

In [26]:
# 나와 타이타닉의 내적 구하기 (선호 리스트에 있는 영화)
np.dot(jenny_vector, titanic_vector)

0.69579905

In [27]:
# 다른 영화에 대한 선호도 예측 (선호 리스트에 없는 영화)
matrix = movie_to_idx['My Fair Lady (1964)']
matrix_vector = als_model.item_factors[matrix]
np.dot(jenny_vector, matrix_vector)

0.05902475

# 7. 내가 좋아하는 영화와 비슷한 영화 추천

In [28]:
favorite_movie = 'Mission: Impossible (1996)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie   # (movie idx, 유사도) Tuple

[(58, 1.0),
 (138, 0.5284145),
 (118, 0.48484734),
 (587, 0.47482374),
 (156, 0.46559694),
 (179, 0.44948187),
 (766, 0.44903842),
 (82, 0.43846905),
 (714, 0.43518576),
 (772, 0.43057784),
 (129, 0.42631635),
 (140, 0.41672072),
 (975, 0.4042716),
 (435, 0.3886269),
 (1943, 0.38709497)]

In [29]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Mission: Impossible (1996)',
 'True Lies (1994)',
 'Con Air (1997)',
 'Batman Returns (1992)',
 'Mask of Zorro, The (1998)',
 'Rock, The (1996)',
 'GoldenEye (1995)',
 'Lost World: Jurassic Park, The (1997)',
 'Waterworld (1995)',
 'Rising Sun (1993)',
 'Conspiracy Theory (1997)',
 'Clear and Present Danger (1994)',
 'Young Sherlock Holmes (1985)',
 'Godfather: Part III, The (1990)',
 'Batman & Robin (1997)']

In [30]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [31]:
get_similar_movie('Gladiator (2000)')

['Gladiator (2000)',
 'Patriot, The (2000)',
 'Mission: Impossible 2 (2000)',
 'Perfect Storm, The (2000)',
 'X-Men (2000)',
 'U-571 (2000)',
 'Frequency (2000)',
 'Shanghai Noon (2000)',
 'Gone in 60 Seconds (2000)',
 'Shaft (2000)']

In [32]:
get_similar_movie('2001: A Space Odyssey (1968)')

['2001: A Space Odyssey (1968)',
 'Close Encounters of the Third Kind (1977)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)',
 'Clockwork Orange, A (1971)',
 '2010 (1984)',
 'Alien (1979)',
 'Brazil (1985)',
 'Blade Runner (1982)',
 'Apocalypse Now (1979)',
 '8MM (1999)']

# 8. 내가 가장 좋아할 만한 영화 추천

In [33]:
user = user_to_idx['jenny']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(50, 0.50636333),
 (39, 0.3608958),
 (126, 0.31682932),
 (248, 0.29198194),
 (330, 0.27225143),
 (4, 0.23439188),
 (626, 0.22994356),
 (60, 0.2287566),
 (139, 0.22874075),
 (110, 0.22863382),
 (479, 0.20951699),
 (384, 0.20842972),
 (255, 0.1984038),
 (322, 0.1898619),
 (33, 0.18564972),
 (160, 0.18369524),
 (143, 0.17946643),
 (99, 0.17829229),
 (121, 0.1776527),
 (641, 0.17341408)]

In [34]:
# 인덱스를 영화 제목으로 변환
[idx_to_movie[i[0]] for i in movie_recommended]

['Toy Story 2 (1999)',
 'Apollo 13 (1995)',
 'Shakespeare in Love (1998)',
 'Good Will Hunting (1997)',
 'Lion King, The (1994)',
 "Bug's Life, A (1998)",
 "You've Got Mail (1998)",
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'October Sky (1999)',
 'Groundhog Day (1993)',
 'Contact (1997)',
 'Jerry Maguire (1996)',
 "There's Something About Mary (1998)",
 'Babe (1995)',
 'Aladdin (1992)',
 'Forrest Gump (1994)',
 'Gone with the Wind (1939)',
 'American Beauty (1999)',
 'Silence of the Lambs, The (1991)',
 "Mr. Holland's Opus (1995)"]

In [35]:
# 사용자가 기록을 남긴 데이터가 특정 영화 추천에 기여한 정도를 확인
recommended = movie_to_idx['Blade Runner (1982)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Toy Story (1995)', 0.015202926284850439),
 ('Home Alone (1990)', -0.002879711428088287),
 ('Titanic (1997)', -0.010803508339650849),
 ('Sixth Sense, The (1999)', -0.022343710851984232),
 ('Truman Show, The (1998)', -0.029618814526831478)]

In [36]:
rec = [idx_to_movie[i[0]] for i in movie_recommended]
recommendation = pd.DataFrame(rec)
recommendation.columns = ['추천 영화 Top 20']
recommendation

,추천 영화 Top 20
0,Toy Story 2 (1999)
1,Apollo 13 (1995)
2,Shakespeare in Love (1998)
3,Good Will Hunting (1997)
4,"Lion King, The (1994)"
5,"Bug's Life, A (1998)"
6,You've Got Mail (1998)
7,Star Wars: Episode I - The Phantom Menace (1999)
8,October Sky (1999)
9,Groundhog Day (1993)


# 회고
- 데이터 전처리 과정에서 영화 평점과 제목이 나뉘어져 있어 두 데이터프레임을 join하게 되었다. 그리고 'timestamp', 'genre'컬럼은 사용하지 않을거라서 보기 편하게 제거했다.
- CSR matrix 생성할때 차원이 초과되어 오류가 발생했다. 좋아하는 영화 5개를 추가하는 부분에서 user_id와 movie_id를 숫자로 추가했는데도 에러가 났다. 알고보니 데이터가 숫자처럼 보이지만 사실 제대로 인덱싱이 되어 있지 않아서 오류가 생긴 것이었다. 실제로 인덱싱 과정을 거치니 matrix가 제대로 생성되었다.
- 결과적으로 선호하는 영화 '타이타닉'에 대한 선호도를 계산했을 때 0.69579905이라는 값을 얻을 수 있었다. 그리고 선호 리스트에 없는 영화에 대한 예측을 했을 때 0.05902475으로 비교적 낮게 나왔다. 
- 유사한 장르의 영화가 추천되어 모델의 성능이 꽤 괜찮은 것 같다.